In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import nibabel as nib
import matplotlib.pyplot as plt
import nibabel.freesurfer.mghformat as mgh

from glob import glob
from os.path import join, exists
from nilearn import plotting, datasets, image

/Users/katherine.b/Library/Python/3.8/lib/python/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
sns.set(style='whitegrid', context='talk')
plt.rcParams["font.family"] = "monospace"
plt.rcParams['font.monospace'] = 'Courier New'

In [3]:
ABCD_DIR = "/Volumes/projects_herting/LABDOCS/PROJECTS/ABCD/Data/release4.0"
PROJ_DIR = "/Volumes/projects_herting/LABDOCS/Personnel/Katie/deltaABCD_clustering/"
DATA_DIR = "data/"
FIGS_DIR = "figures/"
OUTP_DIR = "output/"
RESR_DIR = "resources/"

In [4]:
data_dictionary = pd.read_csv(join(ABCD_DIR, 'generate_dataset/data_element_names.csv'), 
                              header=0, 
                              index_col=0)

In [5]:
data = pd.read_csv(join(PROJ_DIR, DATA_DIR, "data_qcd.csv"), index_col=0, header=0)

In [6]:
smri_vars = pd.concat([data.filter(regex='smri.*change_score'), 
                                data.filter(regex='mrisdp.*change_score')], axis=1).dropna(how='all').columns
rsfmri_vars = data.filter(regex='rsfmri.*change_score').dropna(how='all').columns
rsi_vars = data.filter(regex='dmri_rsi.*change_score').dropna(how='all').columns
dti_vars = data.filter(regex='dmri_dti.*change_score').dropna(how='all').columns

In [7]:
smri_vars = [v.split('.')[0] for v in smri_vars]
rsfmri_vars = [v.split('.')[0] for v in rsfmri_vars]
rsi_vars = [v.split('.')[0] for v in rsi_vars]
dti_vars = [v.split('.')[0] for v in dti_vars]

In [8]:
imaging_vars = list(smri_vars) + list(rsfmri_vars) + list(rsi_vars) + list(dti_vars)

In [9]:
mapping = pd.DataFrame(columns=['modality', 
                                'abcd_structure',
                                'abcd_description', 
                                'atlas_description', 
                                'atlas', 
                                'atlas_value',
                                'atlas_fname'])

In [10]:
destrieux = datasets.fetch_atlas_surf_destrieux()
destrieux_vol = datasets.fetch_atlas_destrieux_2009()
desikan = datasets.fetch_neurovault_ids(image_ids=(23262, ))
#gordon taken care of in 0.0.3
subcort = datasets.fetch_atlas_harvard_oxford('sub-maxprob-thr50-2mm')


Reading local neurovault data.
Already fetched 1 image
1 image found on local disk.


/Users/katherine.b/Library/Python/3.8/lib/python/site-packages/numpy/lib/npyio.py:2405: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  output = genfromtxt(fname, **kwargs)


In [11]:
scs_nifti = subcort.maps

In [12]:
nib.save(scs_nifti, join(PROJ_DIR, RESR_DIR, 'harvox-subcortical-maxprob-thr50-2mm'))

In [13]:
for i in imaging_vars:
    if i in smri_vars:
        mapping.at[i, 'modality'] = 'smri'
    elif i in rsfmri_vars:
        mapping.at[i, 'modality'] = 'fmri' 
    elif i in rsi_vars:
        mapping.at[i, 'modality'] = 'rsi'
    elif i in dti_vars:
        mapping.at[i, 'modality'] = 'dti'
    mapping.at[i, 'abcd_description'] = data_dictionary.loc[i]['description']
    mapping.at[i, 'abcd_structure'] = data_dictionary.loc[i]['structure']
    if '_scs_' in i:
        mapping.at[i, 'atlas'] = 'Subcortical'
        mapping.at[i, 'atlas_fname'] = join(PROJ_DIR, RESR_DIR, 'harvox-subcortical-maxprob-thr50-2mm.nii')
    elif 'gordon' in i:
        mapping.at[i, 'atlas'] = 'Gordon'
        #mapping.at[i, 'atlas_fname'] = 
    elif '_cdk_' in i:
        mapping.at[i, 'atlas'] = 'Desikan'
        mapping.at[i, 'atlas_fname'] = desikan.images[0]
    elif 'mrisdp' in i:
        mapping.at[i, 'atlas'] = 'Destrieux'
        mapping.at[i, 'atlas_fname'] = destrieux_vol.maps
    elif '_cf12_' in i:
        # remove
        mapping.at[i, 'atlas'] = 'Fuzzy 12'
    elif 'fib' in i:
        mapping.at[i, 'atlas'] = 'Fiber Atlas'

In [14]:
l_and_r_destrieux = destrieux.labels * 2
for i in range(0, len(l_and_r_destrieux)):
    if i == 0:
        pass
    elif i < 42:
        mapping.at[f'mrisdp_{i}','atlas_description'] = l_and_r_destrieux[i]
        mapping.at[f'mrisdp_{i}','atlas_value'] = i
        mapping.at[f'mrisdp_{302 + i}','atlas_description'] = l_and_r_destrieux[i]
        mapping.at[f'mrisdp_{302 + i}','atlas_value'] = i
        mapping.at[f'mrisdp_{906 + i}','atlas_description'] = l_and_r_destrieux[i]
        mapping.at[f'mrisdp_{906 + i}','atlas_value'] = i
    elif i == 42:
        pass
    elif 42 < i < 76:
        mapping.at[f'mrisdp_{i - 1}','atlas_description'] = l_and_r_destrieux[i]
        mapping.at[f'mrisdp_{i - 1}','atlas_value'] = i
        mapping.at[f'mrisdp_{302 + i - 1}','atlas_description'] = l_and_r_destrieux[i]
        mapping.at[f'mrisdp_{302 + i - 1}','atlas_value'] = i
        mapping.at[f'mrisdp_{906 + i - 1}','atlas_description'] = l_and_r_destrieux[i]
        mapping.at[f'mrisdp_{906 + i - 1}','atlas_value'] = i
    elif i == 76:
        pass
    elif 76 < i < 116:
        mapping.at[f'mrisdp_{i - 2}','atlas_description'] = l_and_r_destrieux[i]
        mapping.at[f'mrisdp_{i - 2}','atlas_value'] = i
        mapping.at[f'mrisdp_{302 + i - 2}','atlas_description'] = l_and_r_destrieux[i]
        mapping.at[f'mrisdp_{302 + i - 2}','atlas_value'] = i
        mapping.at[f'mrisdp_{906 + i - 2}','atlas_description'] = l_and_r_destrieux[i]
        mapping.at[f'mrisdp_{906 + i - 2}','atlas_value'] = i
    elif i == 116:
        pass
    elif i > 116:
        mapping.at[f'mrisdp_{i - 3}','atlas_description'] = l_and_r_destrieux[i]
        mapping.at[f'mrisdp_{i - 3}','atlas_value'] = i
        mapping.at[f'mrisdp_{302 + i - 3}','atlas_description'] = l_and_r_destrieux[i]
        mapping.at[f'mrisdp_{302 + i - 3}','atlas_value'] = i
        mapping.at[f'mrisdp_{906 + i - 3}','atlas_description'] = l_and_r_destrieux[i]
        mapping.at[f'mrisdp_{906 + i - 3}','atlas_value'] = i

In [15]:
desikan_labels = pd.read_csv(join(PROJ_DIR, DATA_DIR, 'desikan_labels.txt'), 
                             sep='\t', 
                             header=0,
                             usecols=['value']
                            )

In [16]:
cdk_area = mapping.filter(regex='smri_area.*_cdk_.*', axis=0).index
cdk_thick = mapping.filter(regex='smri_thick.*_cdk_.*', axis=0).index
cdk_t1wcnt = mapping.filter(regex='smri_t1wcnt.*_cdk_.*', axis=0).index
cdk_tvar = mapping.filter(regex='rsfmri_var_cdk_.*', axis=0).index
cdk_rnd = mapping.filter(regex='dmri_rsirndgm_cdk.*', axis=0).index
cdk_rni = mapping.filter(regex='dmri_rsirnigm_cdk.*', axis=0).index

In [17]:
for i in range(0, len(desikan_labels.values)):
    mapping.at[cdk_area[i],'atlas_description'] = desikan_labels.iloc[i]['value']
    mapping.at[cdk_area[i],'atlas_value'] = desikan_labels.index[i]
    mapping.at[cdk_thick[i],'atlas_description'] = desikan_labels.iloc[i]['value']
    mapping.at[cdk_thick[i],'atlas_value'] = desikan_labels.index[i]
    mapping.at[cdk_t1wcnt[i],'atlas_description'] = desikan_labels.iloc[i]['value']
    mapping.at[cdk_t1wcnt[i],'atlas_value'] = desikan_labels.index[i]
    mapping.at[cdk_tvar[i],'atlas_description'] = desikan_labels.iloc[i]['value']
    mapping.at[cdk_tvar[i],'atlas_value'] = desikan_labels.index[i]
    mapping.at[cdk_rnd[i],'atlas_description'] = desikan_labels.iloc[i]['value']
    mapping.at[cdk_rnd[i],'atlas_value'] = desikan_labels.index[i]
    mapping.at[cdk_rni[i],'atlas_description'] = desikan_labels.iloc[i]['value']
    mapping.at[cdk_rni[i],'atlas_value'] = desikan_labels.index[i]

In [18]:
scs_1 = ['x',
         'x',
         'x',
         'x',
        "tplh",
        "caudatelh",
        "putamenlh",
        "pallidumlh",
        "bstem",
        "hpuslh",
        "amygdalalh",
        "aal",
        'x',
        'x',
         'x',
        "tprh",
        "caudaterh",
        "putamenrh",
        "pallidumrh",
        "hpusrh",
        "amygdalarh",
        "aar"]

In [19]:
subcort_map = {}
for i in range(0, len(scs_1)):
    if scs_1[i] == 'x':
        pass
    else:
        subcort_map[scs_1[i]] = [subcort.labels[i], i]

In [20]:
subcort_map

{'tplh': ['Left Thalamus', 4],
 'caudatelh': ['Left Caudate', 5],
 'putamenlh': ['Left Putamen', 6],
 'pallidumlh': ['Left Pallidum', 7],
 'bstem': ['Brain-Stem', 8],
 'hpuslh': ['Left Hippocampus', 9],
 'amygdalalh': ['Left Amygdala', 10],
 'aal': ['Left Accumbens', 11],
 'tprh': ['Right Thalamus', 15],
 'caudaterh': ['Right Caudate', 16],
 'putamenrh': ['Right Putamen', 17],
 'pallidumrh': ['Right Pallidum', 18],
 'hpusrh': ['Right Hippocampus', 19],
 'amygdalarh': ['Right Amygdala', 20],
 'aar': ['Right Accumbens', 21]}

In [21]:
all_meas = [i.split('_')[-1] for i in list(mapping.index)]

In [22]:
scs_fc = {'aalh': ['Left Accumbens', 11], 
          'aarh': ['Right Accumbens', 21], 
          'aglh': ['Left Amygdala', 10], 
          'agrh': ['Right Amygdala', 20], 
          'bs': ['Brain-Stem', 8], 
          'cdelh': ['Left Caudate', 5], 
          'cderh': ['Right Caudate', 16], 
          'crcxlh': ['Left Cerebellum', np.nan],
          'crcxrh': ['Right Cerebellum', np.nan], 
          'hplh': ['Left Hippocampus', 9], 
          'hprh': ['Right Hippocampus', 19], 
          'pllh': ['Left Pallidum', 7], 
          'plrh': ['Right Pallidum', 18], 
          'ptlh': ['Left Putamen', 6], 
          'ptrh': ['Right Putamen', 17], 
          'thplh': ['Left Thalamus', 4],
          'thprh': ['Right Thalamus', 15]}

In [23]:
rsi_scs_vars = mapping.filter(regex=f'.*_rsi.*_scs_.*', axis=0).index
rsi_scs = [i.split('_')[-1] for i in rsi_scs_vars]

In [24]:
rsi_scs_map = {'aalh': ['Left Accumbens', 11], 
               'aarh': ['Right Accumbens', 21],
               'ablh': ['Left Accumbens', 11], 
               'abrh': ['Right Accumbens', 21], 
               'aglh': ['Left Amygdala', 10], 
               'agrh': ['Right Amygdala', 20], 
               'bs': ['Brain-Stem', 8], 
               'cbclh': ['Left Cerebellar Cortex', np.nan], 
               'cbcrh': ['Right Cerebellar Cortex', np.nan], 
               'cbwmlh': ['Left Cerebellar White Matter', np.nan],
               'cbwmrh': ['Right Cerebellar White Matter', np.nan], 
               'cdlh': ['Left Caudate', 5], 
               'cdrh': ['Right Caudate', 16], 
               'cdelh': ['Left Caudate', 5], 
               'cderh': ['Right Caudate', 16], 
               'cwmlh': ['Left Cerebral White Matter', np.nan], 
               'cwmrh': ['Right Cerebral White Matter', np.nan], 
               'hclh': ['Left Hippocampus', 9], 
               'hcrh': ['Right Hippocampus', 19],
               'hplh': ['Left Hippocampus', 9], 
               'hprh': ['Right Hippocampus', 19],
               'ilvlh': ['Left Inferior Lateral Ventrical', np.nan], 
               'ilvrh': ['Right Inferior Lateral Ventrical', np.nan], 
               'lvrh': ['Right Lateral Ventrical', np.nan], 
               'pllh': ['Left Pallidum', 7], 
               'plrh': ['Right Pallidum', 18], 
               'ptlh': ['Left Putamen', 6], 
               'ptrh': ['Right Putamen', 17], 
               'tplh': ['Left Thalamus', 4],
               'tprh': ['Right Thalamus', 15],
               'thplh': ['Left Thalamus', 4],
               'thprh': ['Right Thalamus', 15],
               'vdclh': ['Left Ventricle', np.nan], 
               'vdcrh': ['Right Ventricle', np.nan]}

In [25]:
# assign values to subcortical variables
scs_vars = list(mapping.filter(regex='.*_scs_.*', axis=0).index)
for key in scs_vars:
    #print(key)
    scs_key = key.split('_')[-1]
    if '_cor_' in key:
        if not scs_key in rsi_scs_map:
            print(f'{scs_key} not in rsi_scs_map')
            pass
        else:
            print(scs_key, rsi_scs_map[scs_key])
            mapping.at[key,'atlas_description'] = rsi_scs_map[scs_key][0]
            mapping.at[key,'atlas_value'] = rsi_scs_map[scs_key][1]
            #print(key1, scs_fc[key1][1])
    elif 'rsi' in key:
        if not scs_key in scs_fc:
            #print(f'{scs_key} not in scs_fc')
            pass
        else:
            #print(scs_key, scs_fc[scs_key])
            mapping.at[key,'atlas_description'] = scs_fc[scs_key][0]
            mapping.at[key,'atlas_value'] = scs_fc[scs_key][1]
            #print(key1, scs_fc[key1][1])
    else:
        if not scs_key in subcort_map:
            #print(f'{scs_key} not in subcort_map')
            pass
        else:
            #print(var)
            mapping.at[key,'atlas_description'] = subcort_map[scs_key][0]
            mapping.at[key,'atlas_value'] = subcort_map[scs_key][1]

crcxlh not in rsi_scs_map
thplh ['Left Thalamus', 4]
cdelh ['Left Caudate', 5]
ptlh ['Left Putamen', 6]
pllh ['Left Pallidum', 7]
bs ['Brain-Stem', 8]
hplh ['Left Hippocampus', 9]
aglh ['Left Amygdala', 10]
aalh ['Left Accumbens', 11]
vtdclh not in rsi_scs_map
crcxrh not in rsi_scs_map
thprh ['Right Thalamus', 15]
cderh ['Right Caudate', 16]
ptrh ['Right Putamen', 17]
plrh ['Right Pallidum', 18]
hprh ['Right Hippocampus', 19]
agrh ['Right Amygdala', 20]
aarh ['Right Accumbens', 21]
vtdcrh not in rsi_scs_map
crcxlh not in rsi_scs_map
thplh ['Left Thalamus', 4]
cdelh ['Left Caudate', 5]
ptlh ['Left Putamen', 6]
pllh ['Left Pallidum', 7]
bs ['Brain-Stem', 8]
hplh ['Left Hippocampus', 9]
aglh ['Left Amygdala', 10]
aalh ['Left Accumbens', 11]
vtdclh not in rsi_scs_map
crcxrh not in rsi_scs_map
thprh ['Right Thalamus', 15]
cderh ['Right Caudate', 16]
ptrh ['Right Putamen', 17]
plrh ['Right Pallidum', 18]
hprh ['Right Hippocampus', 19]
agrh ['Right Amygdala', 20]
aarh ['Right Accumbens', 21]


In [26]:
mapping.filter(regex=f'.*_rsi.*_scs_.*', axis=0)

,modality,abcd_structure,abcd_description,atlas_description,atlas,atlas_value,atlas_fname
dmri_rsirni_scs_ilvlh,rsi,abcd_drsip101,restricted normalized isotropic diffusion aseg...,NaN,Subcortical,NaN,/Volumes/projects_herting/LABDOCS/Personnel/Ka...
dmri_rsirni_scs_cbwmlh,rsi,abcd_drsip101,restricted normalized isotropic diffusion aseg...,NaN,Subcortical,NaN,/Volumes/projects_herting/LABDOCS/Personnel/Ka...
dmri_rsirni_scs_cbclh,rsi,abcd_drsip101,restricted normalized isotropic diffusion aseg...,NaN,Subcortical,NaN,/Volumes/projects_herting/LABDOCS/Personnel/Ka...
dmri_rsirni_scs_tplh,rsi,abcd_drsip101,restricted normalized isotropic diffusion aseg...,NaN,Subcortical,NaN,/Volumes/projects_herting/LABDOCS/Personnel/Ka...
dmri_rsirni_scs_cdlh,rsi,abcd_drsip101,restricted normalized isotropic diffusion aseg...,NaN,Subcortical,NaN,/Volumes/projects_herting/LABDOCS/Personnel/Ka...
dmri_rsirni_scs_ptlh,rsi,abcd_drsip101,restricted normalized isotropic diffusion aseg...,Left Putamen,Subcortical,6,/Volumes/projects_herting/LABDOCS/Personnel/Ka...
dmri_rsirni_scs_pllh,rsi,abcd_drsip101,restricted normalized isotropic diffusion aseg...,Left Pallidum,Subcortical,7,/Volumes/projects_herting/LABDOCS/Personnel/Ka...
dmri_rsirni_scs_bs,rsi,abcd_drsip101,restricted normalized isotropic diffusion aseg...,Brain-Stem,Subcortical,8,/Volumes/projects_herting/LABDOCS/Personnel/Ka...
dmri_rsirni_scs_hclh,rsi,abcd_drsip101,restricted normalized isotropic diffusion aseg...,NaN,Subcortical,NaN,/Volumes/projects_herting/LABDOCS/Personnel/Ka...
dmri_rsirni_scs_aglh,rsi,abcd_drsip101,restricted normalized isotropic diffusion aseg...,Left Amygdala,Subcortical,10,/Volumes/projects_herting/LABDOCS/Personnel/Ka...


In [27]:
# gordon parcellation
gordon_labels = pd.read_excel(join(PROJ_DIR, 'resources', 'gordon', 'Parcels.xlsx'),
                              header=0, 
                              index_col=0)

/Users/katherine.b/Library/Python/3.8/lib/python/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [28]:
cortgordon = mapping.filter(regex='.*_cortgordon', axis=0).index
for var in cortgordon:
    value = int(var.split('_')[-1][2:-2])
    mapping.at[var, 'atlas_value'] = value
    mapping.at[var, 'atlas_description'] = gordon_labels.loc[value]['Community']

region_names = {'aal': ('Accumbens', 'L'), 
                'aalh': ('Accumbens', 'L'), 
                'aar': ('Accumbens', 'R'), 
                'aarh': ('Accumbens', 'R'), 
                'ablh': ('Accumbens', 'L'), 
                'abrh' ('Accumbens', 'R'),
                'ad': ('Auditory Network', 'B'), 
                'aglh': ('Amygdala', 'L'), 
                'agrh': ('Amygdala', 'R'), 
                'amygdalalh': ('Amugdala', 'L'), 
                'amygdalarh': ('Amygdala', 'R'), 
                'aomtmlh': (),
                'aomtmrh': (), 
                'atrlh', 
                'atrrh', 
                'banksstslh': ('Banks of Superior Temporal Sulcus', 'L'), 
                'banksstsrh': ('Banks of Superior Temporal Sulcus', 'R'),
                'brainstem': ('Brainstem', 'B'), 
                'bs': ('Brainstem', 'B'), 
                'bstem': ('Brainstem', 'B'), 
                'ca': ('Cinguloparietal Network', 'B'), 
                'caudatelh': ('Caudate', 'L'), 
                'caudaterh': ('Caudate', 'R'),
                'cbclh': ('Cerebellar Cortex', 'L'), 
                'cbcrh': ('Cerebellar Cortex', 'R'), 
                'cbwmlh': ('Cerebellar White Matter', 'L'), 
                'cbwmrh': ('Cerebellar White Matter', 'R'), 
                'cc': ('Corpus Callosum', 'B'), 
                'cdacatelh': ('Anterior Cingulate, Caudal', 'L'),
                'cdacaterh': ('Anterior Cingulate, Caudal', 'R'), 
                'cdaclatelh': ('Anterior Cingulate, Caudal', 'L'), 
                'cdaclaterh': ('Anterior Cingulate, Caudal', 'R'), 
                'cdelh': ('Caudate', 'L'), 
                'cderh': ('Caudate', 'R'), 
                'cdlh': ('Caudate', 'L'),
                'cdmdflh': ('Middle Frontal Gyrus, Caudal', 'L'), 
                'cdmdfrh': ('Middle Frontal Gyrus, Caudal', 'R'), 
                'cdmdfrlh': ('Middle Frontal Gyrus, Caudal', 'L'), 
                'cdmdfrrh': ('Middle Frontal Gyrus, Caudal', 'R'), 
                'cdrh': ('Caudate', 'R'), 
                'cgc': ('Cingulo-Opercular Network', 'B'),
                'cgclh': ('Cingulate Cingulum', 'L'), 
                'cgcrh': ('Cingulate Cingulum', 'R'), 
                'cghlh': ('Parahippocampal Cingulum', 'L'), 
                'cghrh': ('Parahippocampal Cingulum', 'R'),  
                'crbcortexlh': ('Cerebellar Cortex', 'L'),
                'crbcortexrh': ('Cerebellar Cortex', 'R'), 
                'crbwmatterlh': ('Cerebellar White Matter', 'L'), 
                'crbwmatterrh': ('Cerebellar White Matter', 'L'), 
                'crcxlh': ('Cerebellar Cortex', 'L'), 
                'crcxrh': ('Cerebellar Cortex', 'R'),
                'cstlh': ('Corticospinal Tract', 'L'), 
                'cstrh': ('Corticospinal Tract', 'R'), 
                'cuneuslh': ('Cuneus', 'L'), 
                'cuneusrh': ('Cuneus', 'R'), 
                'cwmlh': ('Cerebral White Matter', 'L'), 
                'cwmrh': ('Cerebral White Matter', 'L'), 
                'dla': ('Dorsal Attention Network', 'B'),
                'dlprefrlh': ('Dorsal Prefrontal Cortex', 'L'), 
                'dlprefrrh': ('Dorsal Prefrontal Cortex', 'R'), 
                'dmfrlh': ('Dorsomedial Frontal Cortex', 'L'), 
                'dmfrrh': ('Dorsomedial Frontal Cortex', 'R'), 
                'dt': ('Default Mode Network', 'B'), 
                'ehinallh': ('Entorhinal Cortex', 'L'),
                'ehinalrh': ('Entorhinal Cortex', 'R'), 
                'entorhinallh': ('Entorhinal Cortex', 'L'), 
                'entorhinalrh': ('Entorhinal Cortex', 'R'), 
                'fflh': ('Fusiform Gyrus', 'L'), 
                'ffrh': ('Fusiform Gyrus', 'R'), 
                'fmaj': ('Fornix Major', 'B'),
                'fmin': ('Fornix Minor', 'B'), 
                'fo': ('Frontoparietal Network', 'B'), 
                'fpolelh': ('Frontal Pole', 'L'), 
                'fpolerh': ('Frontal Pole', 'R'), 
                'frpolelh': ('Frontal Pole', 'L'), 
                'frpolerh': ('Frontal Pole', 'R'),
                'fscslh': ('Superior Corticostriate Tract (Frontal)', 'L'), 
                'fscsrh': ('Superior Corticostriate Tract (Frontal)', 'R'), 
                'fusiformlh': ('Fusiform Gyrus', 'L'), 
                'fusiformrh': ('Fusiform Gyrus', 'R'), 
                'fxcutlh': ('Fornix (excluding Fimbria)', 'L'),
                'fxcutrh': ('Fornix (excluding Fimbria)', 'R'), 
                'fxlh': ('Fornix', 'L'), 
                'fxrh': ('Fornix', 'R'), 
                'hclh': ('Hippocampus', 'L'),
                'hcrh': ('Hippocampus', 'R'), 
                'hplh': ('Hippocampus', 'L'), 
                'hprh': ('Hippocampus', 'R'), 
                'hpuslh': ('Hippocampus', 'L'), 
                'hpusrh': ('Hippocampus', 'R'), 
                'ifolh': ('Inferior Fronto-occipital Fasciculus', 'L'), 
                'iforh': ('Inferior Fronto-occipital Fasciculus', 'R'),
                'ifpalh': ('Inferior Parietal', 'L'), 
                'ifparh': ('Inferior Parietal', 'R'), 
                'ifpllh': ('Inferior Parietal', 'L'), 
                'ifplrh': ('Inferior Parietal', 'R'), 
                'ifsfclh', 
                'ifsfcrh',
       'iftlh', 'iftmlh', 'iftmrh', 'iftrh', 'ihcatelh', 'ihcaterh',
       'ihclatelh', 'ihclaterh', 'ilflh', 'ilfrh', 'ilvlh', 'ilvrh',
       'insulalh', 'insularh', 'intracranialv', 'linguallh', 'lingualrh',
       'lobfrlh', 'lobfrrh', 'loboflh', 'lobofrh', 'loccipitallh',
       'loccipitalrh', 'locclh', 'loccrh', 'lvrh', 'mdtlh', 'mdtmlh',
       'mdtmrh', 'mdtrh', 'mobfrlh', 'mobfrrh', 'moboflh', 'mobofrh', 'n',
       'obfrlh', 'obfrrh', 'occlh', 'occrh', 'pallidumlh', 'pallidumrh',
       'paracentrallh', 'paracentralrh', 'paracnlh', 'paracnrh',
       'parahpallh', 'parahpalrh', 'parsobalislh', 'parsobalisrh',
       'parsobislh', 'parsobisrh', 'parsopclh', 'parsopcrh', 'parsopllh',
       'parsoplrh', 'parstgrislh', 'parstgrisrh', 'parstularislh',
       'parstularisrh', 'pclh', 'pcrh', 'pericclh', 'periccrh', 'pllh',
       'plrh', 'postcentrallh', 'postcentralrh', 'postcnlh', 'postcnrh',
       'precentrallh', 'precentralrh', 'precnlh', 'precnrh',
       'precuneuslh', 'precuneusrh', 'psclatelh', 'psclaterh', 'pscslh',
       'pscsrh', 'pslflh', 'pslfrh', 'ptcatelh', 'ptcaterh', 'ptlh',
       'ptoltmlh', 'ptoltmrh', 'ptrh', 'putamenlh', 'putamenrh',
       'rlaclatelh', 'rlaclaterh', 'rlmdflh', 'rlmdfrh', 'rracatelh',
       'rracaterh', 'rrmdfrlh', 'rrmdfrrh', 'rspltp', 'sa', 'scslh',
       'scsrh', 'sifclh', 'sifcrh', 'slflh', 'slfrh', 'smh', 'smlh',
       'smm', 'smrh', 'spetallh', 'spetalrh', 'suflh', 'sufrh', 'sufrlh',
       'sufrrh', 'supllh', 'suplrh', 'sutlh', 'sutmlh', 'sutmrh', 'sutrh',
       'thplh', 'thprh', 'tmpolelh', 'tmpolerh', 'total', 'tplh',
       'tpolelh', 'tpolerh', 'tprh', 'trvtmlh', 'trvtmrh', 'tslflh',
       'tslfrh', 'tvtlh', 'tvtrh', 'unclh', 'uncrh', 'vdclh', 'vdcrh',
       'vedclh', 'vedcrh', 'ventraldclh', 'ventraldcrh', 'vs', 'vta',
       'vtdclh', 'vtdcrh'}

In [29]:
# skipping fuzzy12 because the atlases are only available upon email request
# and I doubt I'll use them for plotting anyway
# next up: fiber atlas
FIBER_DIR = '/Volumes/projects_herting/LABDOCS/PROJECTS/ABCD/ABCD_Outcomes/ABCD-brain outcomes/AtlasTrack/'
atlastrack_rois = join(FIBER_DIR, 'fibertract_visualization', 'ABCD Atlas Track ROIs')
fiber_key = pd.read_csv(join(FIBER_DIR, 'documentation', 'DTI_Fiber_Legend.csv'), 
                        header=0, 
                        index_col=0)

In [30]:
fiber_key['FiberName'] = [name.lower() for name in fiber_key['FiberName']]

In [31]:
for var in mapping.filter(regex='.*_fiber', axis=0).index:
    tract_name = var.split('_')[-1]
    hemisphere = tract_name[-2]
    if not 'h' in var:
        tract = tract_name
        index = fiber_key[fiber_key['FiberName'] == tract].index
    else:
        tract = tract_name[:-2]
        index = fiber_key[fiber_key['FiberName'] == f'{hemisphere}_{tract}'].index
        mapping.at[var, 'atlas_description'] = fiber_key.loc[index]['LongFiberName']
        mapping.at[var, 'atlas_value'] = int(index.values)
        mapping.at[var, 'atlas_fname'] = join(atlastrack_rois, f'fiber_{int(index.values)}_count_countatlas.nii.gz')

In [32]:
for var in mapping.filter(regex='rsi.*_fib.*', axis=0).index:
    tract_name = var.split('_')[-1]
    hemisphere = tract_name[-2]
    if not 'h' in var:
        tract = tract_name
        index = fiber_key[fiber_key['FiberName'] == tract].index
    else:
        tract = tract_name[:-2]
        index = fiber_key[fiber_key['FiberName'] == f'{hemisphere}_{tract}'].index
        mapping.at[var, 'atlas_description'] = fiber_key.loc[index]['LongFiberName']
        mapping.at[var, 'atlas_value'] = int(index.values)
        mapping.at[var, 'atlas_fname'] = join(atlastrack_rois, f'fiber_{int(index.values)}_count_countatlas.nii.gz')

In [33]:
mapping.to_csv(join(PROJ_DIR, DATA_DIR, 'variable_to_nifti_mapping.csv'))

In [34]:
mapping.filter(regex='cor.*_scs.*', axis=0)

,modality,abcd_structure,abcd_description,atlas_description,atlas,atlas_value,atlas_fname
rsfmri_cor_ngd_au_scs_crcxlh,fmri,mrirscor02,Average correlation between auditory network a...,NaN,Subcortical,NaN,/Volumes/projects_herting/LABDOCS/Personnel/Ka...
rsfmri_cor_ngd_au_scs_thplh,fmri,mrirscor02,Average correlation between auditory network a...,Left Thalamus,Subcortical,4,/Volumes/projects_herting/LABDOCS/Personnel/Ka...
rsfmri_cor_ngd_au_scs_cdelh,fmri,mrirscor02,Average correlation between auditory network a...,Left Caudate,Subcortical,5,/Volumes/projects_herting/LABDOCS/Personnel/Ka...
rsfmri_cor_ngd_au_scs_ptlh,fmri,mrirscor02,Average correlation between auditory network a...,Left Putamen,Subcortical,6,/Volumes/projects_herting/LABDOCS/Personnel/Ka...
rsfmri_cor_ngd_au_scs_pllh,fmri,mrirscor02,Average correlation between auditory network a...,Left Pallidum,Subcortical,7,/Volumes/projects_herting/LABDOCS/Personnel/Ka...
...,...,...,...,...,...,...,...
rsfmri_cor_ngd_vs_scs_plrh,fmri,mrirscor02,Average correlation between visual network and...,Right Pallidum,Subcortical,18,/Volumes/projects_herting/LABDOCS/Personnel/Ka...
rsfmri_cor_ngd_vs_scs_hprh,fmri,mrirscor02,Average correlation between visual network and...,Right Hippocampus,Subcortical,19,/Volumes/projects_herting/LABDOCS/Personnel/Ka...
rsfmri_cor_ngd_vs_scs_agrh,fmri,mrirscor02,Average correlation between visual network and...,Right Amygdala,Subcortical,20,/Volumes/projects_herting/LABDOCS/Personnel/Ka...
rsfmri_cor_ngd_vs_scs_aarh,fmri,mrirscor02,Average correlation between visual network and...,Right Accumbens,Subcortical,21,/Volumes/projects_herting/LABDOCS/Personnel/Ka...


In [35]:
destrieux_vol_nimg = nib.load(destrieux_vol.maps)
destrieux_vol_arra = destrieux_vol_nimg.get_fdata()

In [36]:
desikan_vol_nimg = nib.load(desikan.images[0])
desikan_vol_arra = desikan_vol_nimg.get_fdata()